# 🚀 Memory Management - Part 1 - Sessions (Kaggle Day 3)

**Use this exact order. Run one cell at a time, top → bottom.**

**Tip:** Do **not** use *Run all*. Kaggle may rate-limit (429).

---

## ⚙️ 1) Setup
### 1.1 Install (Kaggle already has `google-adk`) — *skip install*
### 1.2 Configure Gemini API Key via Kaggle Secrets
In the Kaggle top menu: **Add-ons → Secrets** → add label `GOOGLE_API_KEY` with your key and tick the checkbox.

In [ ]:
import os
from kaggle_secrets import UserSecretsClient
try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(f"🔑 Authentication Error: Add 'GOOGLE_API_KEY' in Kaggle Secrets. Details: {e}")

### 1.3 Import ADK components

In [ ]:
from typing import Any, Dict

from google.adk.agents import Agent, LlmAgent
from google.adk.apps.app import App, EventsCompactionConfig
from google.adk.models.google_llm import Gemini
from google.adk.sessions import DatabaseSessionService, InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools.tool_context import ToolContext
from google.genai import types

print("✅ ADK components imported successfully.")

### 1.4 Helper: `run_session(runner, queries, session_name)`

In [ ]:
async def run_session(runner_instance: Runner, user_queries: list[str] | str = None, session_name: str = "default"):
    print(f"\n ### Session: {session_name}")
    app_name = runner_instance.app_name
    # choose any user id for demo
    global USER_ID; USER_ID = "default"
    # session service is attached to runner
    session_service = runner_instance.session_service
    try:
        session = await session_service.create_session(app_name=app_name, user_id=USER_ID, session_id=session_name)
    except:
        session = await session_service.get_session(app_name=app_name, user_id=USER_ID, session_id=session_name)

    if user_queries:
        if isinstance(user_queries, str):
            user_queries = [user_queries]
        for query in user_queries:
            print(f"\nUser > {query}")
            query = types.Content(role="user", parts=[types.Part(text=query)])
            async for event in runner_instance.run_async(user_id=USER_ID, session_id=session.id, new_message=query):
                if event.content and event.content.parts:
                    text = event.content.parts[0].text
                    if text and text != "None":
                        print(f"{runner_instance.agent.model.model if hasattr(runner_instance, 'agent') else 'model'} > ", text)
    else:
        print("No queries!")
print("✅ Helper defined.")

### 1.5 Retry options

In [ ]:
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)
print("✅ Retry config ready.")

---
## 🤹 2) Session Management (In-Memory)
Create a stateful agent whose history persists only while the kernel is alive.

In [ ]:
APP_NAME = "default"
USER_ID = "default"
MODEL_NAME = "gemini-2.5-flash-lite"

root_agent = Agent(
    model=Gemini(model=MODEL_NAME, retry_options=retry_config),
    name="text_chat_bot",
    description="A text chatbot",
)
session_service = InMemorySessionService()
runner = Runner(agent=root_agent, app_name=APP_NAME, session_service=session_service)
print("✅ Stateful agent (in-memory) ready.")

In [ ]:
# 2.5 Test: same session remembers name
await run_session(
    runner,
    [
        "Hi, I am Sam! What is the capital of United States?",
        "Hello! What is my name?",
    ],
    "stateful-agentic-session",
)

---
## 📈 3) Persistent Sessions (SQLite via `DatabaseSessionService`)

In [ ]:
chatbot_agent = LlmAgent(
    model=Gemini(model=MODEL_NAME, retry_options=retry_config),
    name="text_chat_bot",
    description="A text chatbot with persistent memory",
)
db_url = "sqlite:///my_agent_data.db"
session_service = DatabaseSessionService(db_url=db_url)
runner = Runner(agent=chatbot_agent, app_name=APP_NAME, session_service=session_service)
print("✅ Upgraded to persistent sessions (SQLite).")

In [ ]:
# 3.3 Test run 1
await run_session(
    runner,
    [
        "Hi, I am Sam! What is the capital of the United States?",
        "Hello! What is my name?",
    ],
    "test-db-session-01",
)

In [ ]:
# 3.4 Test run 2 (simulates after restart; here we just reuse)
await run_session(
    runner,
    [
        "What is the capital of India?",
        "Hello! What is my name?",
    ],
    "test-db-session-01",
)

In [ ]:
# 3.5 Isolation check
await run_session(
    runner,
    ["Hello! What is my name?"],
    "test-db-session-02",
)


In [ ]:
# 3.6 Peek into SQLite events table
import sqlite3
with sqlite3.connect("my_agent_data.db") as connection:
    cursor = connection.cursor()
    res = cursor.execute("select app_name, session_id, author, content from events")
    print([_[0] for _ in res.description])
    for row in res.fetchall():
        print(row)

---
## ⏳ 4) Context Compaction (Experimental)

In [ ]:
research_app_compacting = App(
    name="research_app_compacting",
    root_agent=chatbot_agent,
    events_compaction_config=EventsCompactionConfig(
        compaction_interval=3,
        overlap_size=1,
    ),
)
session_service = DatabaseSessionService(db_url="sqlite:///my_agent_data.db")
research_runner_compacting = Runner(app=research_app_compacting, session_service=session_service)
print("✅ Research app with compaction ready (experimental warning may appear).")

In [ ]:
# 4.2 Run turns to trigger compaction after 3rd invocation
await run_session(research_runner_compacting, "What is the latest news about AI in healthcare?", "compaction_demo")
await run_session(research_runner_compacting, "Are there any new developments in drug discovery?", "compaction_demo")
await run_session(research_runner_compacting, "Tell me more about the second development you found.", "compaction_demo")
await run_session(research_runner_compacting, "Who are the main companies involved in that?", "compaction_demo")

In [ ]:
# 4.3 Verify compaction event exists
final_session = await session_service.get_session(
    app_name=research_runner_compacting.app_name,
    user_id=USER_ID,
    session_id="compaction_demo",
)
print("--- Searching for Compaction Summary Event ---")
found = False
for ev in final_session.events:
    if ev.actions and getattr(ev.actions, 'compaction', None):
        print("\n✅ SUCCESS! Found the Compaction Event:")
        print(f"  Author: {ev.author}")
        print(f"  Compaction keys: {list(ev.actions.compaction.keys())}")
        found = True
        break
if not found:
    print("❌ No compaction event found. Try running one more turn in the same session.")

---
## 🤝 5) Session State Tools (save/retrieve)

In [ ]:
USER_NAME_SCOPE_LEVELS = ("temp", "user", "app")

def save_userinfo(tool_context: ToolContext, user_name: str, country: str) -> Dict[str, Any]:
    tool_context.state["user:name"] = user_name
    tool_context.state["user:country"] = country
    return {"status": "success"}

def retrieve_userinfo(tool_context: ToolContext) -> Dict[str, Any]:
    user_name = tool_context.state.get("user:name", "Username not found")
    country = tool_context.state.get("user:country", "Country not found")
    return {"status": "success", "user_name": user_name, "country": country}

print("✅ Tools created.")

In [ ]:
APP_NAME = "default"
USER_ID = "default"
MODEL_NAME = "gemini-2.5-flash-lite"

root_agent = LlmAgent(
    model=Gemini(model=MODEL_NAME, retry_options=retry_config),
    name="text_chat_bot",
    description=(
        """A text chatbot.\nTools for managing user context:\n"
        "* To record username and country when provided use `save_userinfo`.\n"
        "* To fetch username and country when required use `retrieve_userinfo`.\n"""
    ),
    tools=[save_userinfo, retrieve_userinfo],
)
session_service = InMemorySessionService()
runner = Runner(agent=root_agent, session_service=session_service, app_name=APP_NAME)
print("✅ Agent with session-state tools initialized.")

In [ ]:
# 5.3 Demo: save + recall
await run_session(
    runner,
    [
        "Hi there, how are you doing today? What is my name?",
        "My name is Sam. I'm from Poland.",
        "What is my name? Which country am I from?",
    ],
    "state-demo-session",
)

In [ ]:
# 5.4 Inspect stored state
session = await session_service.get_session(app_name=APP_NAME, user_id=USER_ID, session_id="state-demo-session")
print("Session State Contents:")
print(session.state)

In [ ]:
# 5.5 New session (isolation demo)
await run_session(
    runner,
    ["Hi there, how are you doing today? What is my name?"],
    "new-isolated-session",
)

In [ ]:
# 5.6 Check state of new session (may appear shared depending on impl)
session = await session_service.get_session(app_name=APP_NAME, user_id=USER_ID, session_id="new-isolated-session")
print("New Session State:")
print(session.state)

---
## 🧹 Cleanup (optional)

In [ ]:
import os
if os.path.exists("my_agent_data.db"):
    os.remove("my_agent_data.db")
print("✅ Cleaned up old database files")